In [31]:
import numpy as np 
import tensorflow as tf
import pandas as pd
#tqdm işlemlerin ilerleme çubuğunu göstermek için kullanılıyor.
from tqdm import tqdm
#dosya dizin işlemleri
import os
#opencv bilgisayarla görme işlemleri için kullanılıyor.
from cv2 import imread, createCLAHE 
import cv2
#dosya aramak 
from glob import glob
#çizim
import matplotlib.pyplot as plt

#dosyalar
image_path = "lung/data/Lung Segmentation/CXR_png"
mask_path = "lung/data/Lung Segmentation/masks"

#görüntü ve maske verilerini okuyup liste halinde döndürüyor
#data hazırlığı
def getData(X_shape):
    im_array = []
    mask_array = []
    
    #dizinindeki tüm dosya adları alınıyor.
    images = os.listdir(image_path)
    
    for i in tqdm(images):  
        # Görüntüyü oku
        image_file = os.path.join(image_path, i)
        if not os.path.exists(image_file):
            #print(f"Warning: Image file not found: {image_file}")
            continue
        #Dosya okunup, belirtilen boyuta (X_shape x X_shape) yeniden boyutlandırılıyor 
        #tek bir kanal (grayscale) olarak alınıyor ([:, :, 0]).
        im = cv2.resize(cv2.imread(image_file), (X_shape, X_shape))[:, :, 0]
        
        # Maskeyi oku
        mask_file = os.path.join(mask_path, os.path.splitext(i)[0] + "_mask.png")
        if not os.path.exists(mask_file):
            continue
        mask = cv2.resize(cv2.imread(mask_file), (X_shape, X_shape))[:, :, 0]
        
        im_array.append(im)
        mask_array.append(mask)
    
    print("Test images length:", len(im_array))
    print("Test masks length:", len(mask_array))
    
    return im_array, mask_array

im_array, mask_array = getData(256)


100%|█████████████████████████████████████████| 800/800 [01:22<00:00,  9.73it/s]

Test images length: 704
Test masks length: 704


In [32]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(im_array, mask_array, test_size = 0.2, random_state= 42)

#numpy dizisine dönüştür
images = np.array(im_array)
mask = np.array(mask_array)

In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

#grayscale
def unet(input_size=(256,256,1)):
    inputs = Input(input_size)

    #encoder
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    #encoder
    
    #bottleneck
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    #bottleneck
    
    #decoder
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    #decoder
    
    #output
    #çıktının boyutunu 1'e indirir 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    
    #modelin giris cikis katmanini döndürür
    return Model(inputs=[inputs], outputs=[conv10])

In [34]:
from tensorflow.keras.losses import binary_crossentropy

model = unet(input_size=(256,256,1))

#her pikselin doğru sınıfına ne kadar yaklaştığını ölçer 0-1 binary
model.compile(optimizer=Adam(), loss='binary_crossentropy', 
              metrics=['accuracy'])

#model.summary()

In [52]:
from IPython.display import clear_output
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split

# Veri setini eğitim ve test setlerine ayırma
train_vol, test_vol, train_seg, test_seg = train_test_split(images / 255.0, (mask > 127).astype(np.float32), 
                                                            test_size=0.2, random_state=127)

# Modeli derleme
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(x=train_vol, y=train_seg, batch_size=8, epochs=5)

# Modeli test etme
test_loss, test_accuracy = model.evaluate(test_vol, test_seg)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Epoch 1/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.9510 - loss: 0.1613
Epoch 2/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.9713 - loss: 0.0738
Epoch 3/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.9725 - loss: 0.0710
Epoch 4/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.9720 - loss: 0.0726
Epoch 5/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.9759 - loss: 0.0631
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9769 - loss: 0.0643
Test Loss: 0.06820625811815262
Test Accuracy: 0.9759278893470764


In [55]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Görsel yolu
img_path = 'ornek.png'

# Görseli yükleme ve model girişi için hazırlama
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)

# Tahminleri eşikleme
threshold = 0.5
predictions[predictions > threshold] = 1
predictions[predictions <= threshold] = 0

plt.figure(figsize=(10, 5))

# Giriş görüntüsü
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title('Giriş Görüntü')
plt.axis('off')

# Tahmin edilen maskeler
plt.subplot(1, 2, 2)
plt.imshow(np.squeeze(predictions), cmap='gray')
plt.title('Tahmin Edilen Maske')
plt.axis('off')

plt.show()


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv2d_76" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 256, 256, 3)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(1, 256, 256, 3), dtype=float32)
  • training=False
  • mask=None